# Projet SOIA 6.1 - PINN
## Oscar Agnus, SOIA A3


Here, we try to simulate the 1D heat equation using a physics-informed neural network (PINN). The heat equation is given by:
$$
\frac{\partial u}{\partial t} = \nu \frac{\partial^2 u}{\partial x^2}
$$

where $u$ is the temperature,
$x$ is the space coordinate,
$t$ is the time coordinate,
and $\nu$ is the thermal diffusivity.


In [1]:
# Imports
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time

import pinnde.pde_Solvers as pde_Solvers
import pinnde.pde_Initials as pde_Initials
import pinnde.pde_Boundaries_2var as pde_Boundaries

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

# Weights and Biases - this is for logging the results
import wandb

### Setting up the problem

Here, we define the problem in physical terms, and format it so that it can be used by the neural network.

### Machine learning model

In [2]:
# Define the model
my_model = models.resnet18()
my_model.fc = nn.Linear(512, 1)

# my_model = torch.nn.Sequential(
#     torch.nn.Linear(1, 20),
#     torch.nn.Tanh(),
#     torch.nn.Linear(20, 20),
#     torch.nn.Tanh(),
#     torch.nn.Linear(20, 20),
#     torch.nn.Tanh(),
#     torch.nn.Linear(20, 1)
# )

model_name = 'resnet18_vanilla'

In [3]:
# Space domain
x_min = 0
x_max = 1
N_x = 1000 # Number of points in the space domain
X = np.linspace(x_min, x_max, N_x) # Discretized space domain
X_bc = np.array([x_min, x_max]) # Boundary conditions pointsfqxy 

# Time domain
t_min = 0
t_max = 1
N_t = 500 # Number of points in the time domain
T = np.linspace(t_min, t_max, N_t) # Discretised time domain
T_0 = np.array(T[0]) # Initial time


# Define useful parameters
nu = 0.01 # Thermal diffusivity

N_u = 1000          # Number of data points for the solution
N_bc = 100          # Number of boundary conditions points
N_pde = 10000       # Number of PDE conditions
N_sensors = 30000   # Number of sensors
N_iv = 100          # Number of initial values

# Initial conditions
def u_ic(x):
    """Initial conditions for the heat equation."""
    return np.sin(np.pi*x)

# Boundary conditions
def u_bc(x, t):
    """Boundary conditions for the heat equation.
    These can be time-dependent."""
    u = np.zeros_like(x)
    u[0] = 0
    u[-1] = 0
    return u # Maybe return only the values on the boundary instead of the whole array ?
    

# Exact solution, or the finite difference solution
def u_exact(x, t):
    """Exact solution of the heat equation.
    The solution is given for each instant, and the space domain can be multidimensional.
    :param x: space domain, discretized
    :param t: time domain, discretized"""
    u = np.zeros((len(t), len(x)))
    for i in range(len(t)):
        u[i] = np.exp(-np.pi**2*t[i])*np.sin(np.pi*x)
    return u

Here are defined the loss functions. There are 3 different loss functions:
- $L_{data}$: loss function for the data points
- $L_{bc}$  :loss function for the boundary conditions
- $L_{pde}$ : loss function for the PDE

Each one of them will be used with their relative weight, which represent their importance, in the composite loss function.

In [4]:
# Gradient function
def grad(outputs, inputs):
    """Compute the gradient of 'outputs' with respect to 'inputs'."""
    return tf.gradients(outputs, inputs)[0]
    # return torch.autograd(outputs, inputs, grad_outputs=torch.ones_like(outputs), create_graph=True)[0]

# Loss functions
def loss_bc(u_pred, x_bc, t_bc, u_bc):
    """Loss function for the boundary conditions."""
    u_square = u_pred(x_bc, t_bc) # estimation de u aux bords
    return tf.reduce_mean(tf.square(u_square - u_bc))

def loss_pde(u_pred, x_pde, t_pde):
    """Loss function for the PDE."""
    u_x  = grad(u_pred, x_pde)
    u_xx = grad(u_x, x_pde)
    u_t  = grad(u_pred, t_pde)
    return tf.reduce_mean(tf.square(u_t - nu*u_xx))

def loss_data(u_pred, u_data):
    """Loss function for the data points."""
    return tf.reduce_mean(tf.square(u_pred - u_data))

# Weights for the composite loss function
w_data = 0.0
w_bc   = 1.0
w_pde  = 1.0

# Composite loss function
def loss_composite(u_data, u_pred, x_bc, t_bc, u_bc, x_pde, t_pde):
    """Composite loss function, combining the three loss functions."""
    return w_data*loss_data(u_pred, u_data) + w_bc*loss_bc(u_pred, x_bc, t_bc, u_bc) + w_bc*loss_pde(u_pred, x_pde, t_pde)

Here we define the training and testing functions. We want to make a checkpoint of the model when the testing loss is minimal.

In [5]:
# Training function 
def train(model, trainloader, optimizer, device):
    model.train()
    running_loss = 0.0

    n_iter = 500

    # Training loop
    for n in range(n_iter):
    # for inputs in trainloader:
        # Reshape to [batch_size, channels, height, width]
        # inputs = inputs.reshape(1, 1, 1, 1000)
        # inputs = inputs.view()  # Reshape to [batch_size, channels, height, width]
    
        inputs = torch.tensor(inputs, dtype=torch.float32)


        # Move data to device
        inputs = inputs.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        print("outputs shape :", outputs.shape)
        outputs_bc = outputs(X_bc, T) # estimated u at the boundary
        loss = loss_composite(u_exact(X, T), outputs, X_bc, T, outputs_bc, X, T)
        loss.backward()
        optimizer.step()

        # Compute running statistics
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        
    train_loss = running_loss / len(trainloader)

    return train_loss

# # Testing function
# def test(model, testloader, criterion, device):
#     model.eval()
#     running_loss = 0.0
#     correct = 0
#     total = 0
# 
#     with torch.no_grad():
#         # Testing loop
#         for inputs, labels in testloader:
#             # Move data to device
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
# 
#             # Compute running statistics
#             running_loss += loss.item()
#             _, predicted = outputs.max(1)
#             total += labels.size(0)
#             correct += predicted.eq(labels).sum().item()
# 
#     test_loss = running_loss / len(testloader)
#     test_acc = 100. * correct / total
#     
#     return test_loss, test_acc

### Training the model

Check in the PC configuration if the GPU is available. Cuda needs to be installed to use the GPU with PyTorch.
This will speed up the training process.

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Is cuda available: ', torch.cuda.is_available())
my_model.to(device)
print(f'Using device: {device}')

Is cuda available:  True
Using device: cuda


Here, we load the data and define the data loaders.


There isn't any testing data, because we will not use the accuracy metrics to assess the model.

The training data `u_data` will be either the exact solution, or some measured data. For now, we will not use any data, meaning `w_data = 0.0`.

In [7]:
# Data loaders

# trainloader = np.array(u_exact(X, T))
trainloader = torch.utils.data.DataLoader(X, batch_size=1, shuffle=True)

In [8]:
# Training parameters
n_epoch = 10
lr = 0.01
optimiser = optim.Adam(my_model.parameters(), lr=lr)

# List of losses
train_loss_list = np.zeros(n_epoch)
train_acc_list  = np.zeros(n_epoch)
best_test_acc = 0.0

# Training
for epoch in range(n_epoch):
    t0 = time.time()
    train_loss = train(my_model, trainloader, optimiser, device)
    t1 = time.time()
    
    print(f'Epoch {epoch+1:03d}, Train Loss: {train_loss:.4f}, Time: {t1-t0:.4f} s')

UnboundLocalError: cannot access local variable 'inputs' where it is not associated with a value

Here we initialise the wandb run, which will log the results of the training process.

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="Projet SOIA 6.1 - PINN",

    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "resnet18",
    "dataset": "heat_equation_custom_solution",
    "epochs": n_epoch,
    }
)

In [18]:
# Display the results
